In [11]:
import pytest
import ipytest
import functools
import numpy as np
from warnings import warn
from copy import deepcopy

ipytest.autoconfig()

In [12]:
"""utils module."""
def keyfy(lst):
    """
        @abstract returns array entries in string fashion delimited by commas

        @param {Array} arr
        @return {string} str
    """
    return str(lst).strip('[]').replace(' ', '')


def reduce_(func, elems, elem0):
    """
        @abstract returns reduce function handler

        @param {function} func
        @param {object} elems
        @param {object} elem0
        @return {string} str
    """
    return functools.reduce(func, elems+[elem0])


def unique(lst):
    """
        @abstract returns list with unique elements

        @param {array} arr
        @return {array} sorted_arr
    """
    return list(np.unique(lst))


def delimited_sort(str_, delimiter):
    """
        @abstract returns a sorted string delimited by token

        @param {array} arr
        @return {array} sorted_arr
    """
    return delimiter.join(sorted(str_.split(delimiter)))


def non_empty_sets_keys(sets):
    """
        @abstract returns a set with non-empty values

        @param {dict} set
        @return {dict} cleaned_set
    """

    return list(
        filter(
            lambda key: len(sets[key]) != 0,
            sets.keys(),
        ),
    )


In [13]:
delimiter = ','

def euler(sets):
    '''
        @abstract returns each tuple [key, elems] of the Euler diagram
        systematic in a generator-wise fashion
        Rationale:
           1. Begin with the available sets and their exclusive elements;
           2. Compute complementary elements to current key-set;
           3. In case complementary set-keys AND current set content
           are not empty, continue;
           Otherwise, go to next key-set;
           4. Find the euler diagram on complementary sets;
           5. Compute exclusive combination elements;
           6. In case there are exclusive elements to combination:
           6.a Yield exclusive combination elements;
           6.b Remove exclusive combination elements from current key-set;

        @param {Array} sets
        @return {Array} keys_elems
    '''
    sets_ = deepcopy(sets)

    # There are no sets
    if not isinstance(sets_, (list, dict)):
        msg_1='Ill-conditioned input.'
        msg_2='It must be either a json-like or array of arrays object!'
        raise TypeError(msg_1+msg_2)

    is_unique_set_arr = [
        len(unique(values)) == len(values) for values in sets_.values()
    ]
    if not reduce_(lambda a, b: a and b, is_unique_set_arr, True):
        warn('Each array MUST NOT have duplicates')
        sets = {key: unique(values) for key, values in sets.items()}

    # Only a set
    if len(sets_.values()) == 1:
        key = list(sets_.keys())[0]
        value = list(sets_.values())[0]
        yield (key, value)

    else:
        # Sets with non-empty elements
        set_keys = non_empty_sets_keys(sets_)

        # Traverse the combination lattice
        for set_key in set_keys:
            compl_sets_keys = list(set(set_keys) - {set_key})

            # There are still sets to analyze
            # Morgan Rule: ¬(A & B) = ¬A | ¬B
            if (len(compl_sets_keys) != 0 and len(sets[set_key]) != 0):
                # Complementary sets
                csets = {
                    cset_key: sets_[cset_key] for cset_key in compl_sets_keys
                }

                # Exclusive combination elements
                for comb_str, celements in euler(csets):

                    # Remove current set_key elements
                    comb_excl = list(set(celements)-set(sets_[set_key]))

                    # Non-empty combination exclusivity case
                    if len(comb_excl) != 0:
                        # 1. Exclusive group elements except current analysis set
                        yield (delimited_sort(comb_str, delimiter), comb_excl)

                        # Remove comb_excl elements from its original sets
                        for ckey in comb_str.split(delimiter):
                            sets_[ckey] = list(
                                set(sets_[ckey])-set(comb_excl),
                            )

                    comb_intersec = list(
                        set(celements).intersection(set(sets[set_key])),
                    )

                    if len(comb_intersec) != 0:
                        # 2. Intersection of analysis element and
                        # exclusive group
                        comb_intersec_key = set_key+delimiter+comb_str

                        yield (
                            delimited_sort(comb_intersec_key, delimiter),
                            comb_intersec,
                        )

                        # Remove intersection elements from
                        # current key-set and complementary sets
                        for ckey in comb_str.split(delimiter):
                            sets_[ckey] = list(
                                set(sets_[ckey])-set(comb_intersec),
                            )

                        sets_[set_key] = list(
                            set(sets_[set_key])-set(comb_intersec),
                        )

                    set_keys = non_empty_sets_keys(sets_)

                # 3. Set-key exclusive elements
                if len(sets_[set_key]) != 0:
                    yield (str(set_key), sets_[set_key])

                    sets_[set_key] = []


def spread_euler(sets):
    '''
        @abstract returns Euler diagram dictionary of set-dictionary of
        non-repetitive elements

        @param {Array} sets
        @return {dict} euler_diagram
    '''
    return dict(euler(sets))


In [14]:
%%ipytest

# define the tests
def test_keyfy():
    '''
    tests the key string generation
    '''
    assert keyfy([1, 2, 3]) == '1,2,3'


def test_one_set_euler():
    '''
    tests the reduce function
    '''
    assert reduce_(lambda a, b: a+b, [1, 2], 0) == 3


def test_unique_elems():
    '''
    tests the reduce function
    '''
    assert unique([1, 2, 3, 3]) == [1, 2, 3]


def test_delimited_sort():
    '''
    tests sorting delimited by token
    '''
    assert delimited_sort('4,1,2,3', ',') == '1,2,3,4'


def test_non_empty_sets_keys():
    '''
    tests dict clean with empty values
    '''
    assert non_empty_sets_keys({'a': [1, 2, 3], 'b': []}) == ['a']

def test_euler_iter_1_input():
    '''
    Generates a tuple with key-value
    '''
    assert next(euler({'a': [1, 2]})) == ('a', [1, 2])


def test_euler_iter_2_input():
    '''
    Generates all tuples with key-value
    '''
    eu_fun = euler({'a': [1, 2], 'b': [2, 3]})

    assert next(eu_fun) == ('b', [3])
    assert next(eu_fun) == ('a,b', [2])
    assert next(eu_fun) == ('a', [1])


def test_euler_iter_warning_1item():
    '''
    Raises a warning for duplicated dict values
    '''
    with pytest.warns(UserWarning):
        next(euler({'a': [42, 42]}))


def test_euler_iter_warning_2items():
    '''
    Raises a warning for duplicated dict values
    '''
    with pytest.warns(UserWarning):
        next(euler({'a': [42, 42], 'b': [42, 42]}))


def test_spread_euler_ill_input_str():
    '''
    Raises an Exception for ill-conditioned input as string
    '''
    with pytest.raises(TypeError, match='Ill-conditioned input.'):
        spread_euler('')


def test_spread_euler_ill_input_num():
    '''
    Raises an Exception for ill-conditioned input as number
    '''
    with pytest.raises(TypeError, match='Ill-conditioned input.'):
        spread_euler(1)


def test_spread_euler_1_set():
    '''
    Returns an euler dictionary for 1 valid set
    '''
    assert spread_euler(
        {
            'a': [1, 2, 3]
        }
    ) == {'a': [1, 2, 3]}


def test_spread_euler_2_sets_with_non_exclusivity():
    '''
    Returns an euler dictionary for 2 valid sets
    '''

    assert spread_euler(
        {
            'a': [1],
            'b': [1, 2]
        }
    ) == {'b': [2], 'a,b': [1]}


def test_spread_euler_2_sets():
    '''
    Returns an euler dictionary for 2 valid sets
    '''
    assert spread_euler(
        {
            'a': [1, 2, 3],
            'b': [2, 3, 4]
        }
    ) == {'b': [4], 'a,b': [2, 3], 'a': [1]}


def test_spread_euler_3_sets():
    '''
    Returns an euler dictionary for 3 valid sets
    '''
    assert spread_euler(
        {
            'a': [1, 2, 3],
            'b': [2, 3, 4],
            'c': [3, 4, 5]
        }) == {'a,b': [2], 'b,c': [4], 'a,b,c': [3], 'c': [5], 'a': [1]}


def test_spread_euler_4_sets():
    '''
    Returns an euler dictionary for 4 valid sets
    '''
    assert spread_euler(
        {
            'a': [1, 2, 3],
            'b': [2, 3, 4],
            'c': [3, 4, 5],
            'd': [3, 5, 6]
        }) == {'a,b': [2], 'b,c': [4], 'a,b,c,d': [3], 'c,d': [5], 'd': [6], 'a': [1]}



no tests ran in 0.00s


In [17]:
print(
    spread_euler(
        {
            'a': [1, 2, 3]
        }
    )
)

print(
    spread_euler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4]
        }
    )
)

print(
    spread_euler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5]
        }
    )
)

print(
    spread_euler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5],
            'd': [3, 5, 6]
        }
    )
)

print(
    spread_euler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5],
            'd': [4, 5, 6]
        }
    )
)

{'a': [1, 2, 3]}
{'b': [4], 'a,b': [2, 3], 'a': [1]}
{'c': [5], 'b,c': [4], 'a,b,c': [3], 'a,b': [2], 'a': [1]}
{'d': [6], 'c,d': [5], 'a,b,c,d': [3], 'b,c': [4], 'a,b': [2], 'a': [1]}
{'d': [6], 'c,d': [5], 'b,c,d': [4], 'a,b,c': [3], 'a,b': [2], 'a': [1]}
